In [30]:
from keras.applications.vgg19 import VGG19
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot  as plt
from keras.layers import Input
import numpy as np
from keras.models import Model




In [31]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.stack([x_train]*3, axis=-1)  # Convert to 3 channels
x_test = np.stack([x_test]*3, axis=-1)    # Convert to 3 channels

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

In [32]:
y_test.shape

(10000,)

In [33]:
x_train.ndim

4

In [34]:
x_train.shape

(60000, 28, 28, 3)

In [35]:
# Load the pre-trained VGG16 model without the top layer
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [7]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [8]:
y_test.shape

(10000, 10)

In [37]:
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), mode='constant') 
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), mode='constant')    

model = Model(inputs=vgg19.input, outputs=vgg19.output)

train_features = model.predict(x_train)
test_features = model.predict(x_test)

print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)


313/313 [==============================] - 32s 103ms/step
Train features shape: (60000, 1, 1, 512)
Test features shape: (10000, 1, 1, 512)


In [38]:
train_features.shape

(60000, 1, 1, 512)

In [39]:
test_features.shape

(10000, 1, 1, 512)

In [40]:
# Flatten the extracted features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [41]:
y_test.shape

(10000,)

In [42]:
y_train.shape

(60000,)

In [43]:
test_features.shape

(10000, 512)

In [44]:
train_features.shape [1]

512

In [45]:
test_features.shape[1]

512

In [46]:
# Build a classifier on top of the extracted features
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [47]:
classifier = Sequential()
classifier.add(Dense(256, activation='relu', input_dim=train_features.shape[1]))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(10, activation='softmax'))

In [50]:
# Compile and train the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
import tensorflow as tf
classifier.fit(train_features, tf.keras.utils.to_categorical(y_train, num_classes=10), batch_size=32, epochs=10, validation_data=(test_features, tf.keras.utils.to_categorical(y_test, num_classes=10)))

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3059 - accuracy: 0.9007 - val_loss: 0.1708 - val_accuracy: 0.9455
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1561 - accuracy: 0.9485 - val_loss: 0.2094 - val_accuracy: 0.9322
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1360 - accuracy: 0.9557 - val_loss: 0.1569 - val_accuracy: 0.9476
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1229 - accuracy: 0.9594 - val_loss: 0.1132 - val_accuracy: 0.9641
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1143 - accuracy: 0.9628 - val_loss: 0.1224 - val_accuracy: 0.9611
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1045 - accuracy: 0.9652 - val_loss: 0.1082 - val_accuracy: 0.9650
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1000 - accuracy: 0.9673 - val_loss: 0.1216 - val_accuracy:

In [52]:
classifier.evaluate(test_features, tf.keras.utils.to_categorical(y_test, num_classes=10))

313/313 [==============================] - 1s 2ms/step - loss: 0.0992 - accuracy: 0.9679


[0.0991513580083847, 0.9678999781608582]